## Lagos Air Quality Monitoring System GUI

This code creates a graphical user interface (GUI) for the "Lagos Air Quality Monitoring System." The main features of the code include:

- Connecting to a Cassandra database to fetch air quality data.
- Allowing the user to select a specific year, attribute, and plot type for visualization.
- Utilizing the Tkinter library to build the GUI components.
- Utilizing Matplotlib for generating and displaying plots based on user selections.
- Providing detailed inline documentation to explain each section of the code.

The GUI application enables users to interact with the air quality data effectively, visualizing it in different ways based on their preferences. By selecting a year, an attribute, and a plot type, users can gain insights into the air quality trends and patterns in the Lagos region. The combination of Cassandra, Tkinter, and Matplotlib makes this code a comprehensive tool for exploring and understanding air quality data.


In [1]:
# Import necessary packages
from tkinter import Tk, Label, StringVar, OptionMenu, Button, Frame
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# Define the Lagos Air Quality Monitoring System GUI application
class LagosAirQualityApp:
    def __init__(self, root):
        self.root = root
        self.root.title('LAGOS AIR QUALITY MONITORING SYSTEM')

        # Load Cassandra credentials from JSON file
        with open("DCBDMS_LAG-token.json") as f:
            secrets = json.load(f)

        CLIENT_ID = secrets["clientId"]
        CLIENT_SECRET = secrets["secret"]

        # Connect to Cassandra cluster using secure connect bundle
        cloud_config = {
            'secure_connect_bundle': 'secure-connect-dcbdms-lag.zip'
        }
        auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
        cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
        self.session = cluster.connect()

        # Update the keyspace and table name accordingly
        self.keyspace = 'dcbdms_keyspace'
        self.table_name = 'lagairdf'

        # Build and execute the query to fetch data from Cassandra
        query = f"SELECT * FROM {self.keyspace}.{self.table_name}"
        rows = self.session.execute(query)

        # Convert the rows to a list of dictionaries
        data = []
        for row in rows:
            data.append(row._asdict())  # Convert Row to dictionary

        # Convert the list of dictionaries to a pandas DataFrame
        self.df = pd.DataFrame(data)

        # Close the Cassandra session and cluster connection
        self.session.shutdown()
        cluster.shutdown()

        # Create GUI components
        self.create_ui()

    def create_ui(self):
        # Year selection
        year_label = Label(self.root, text='Select Year:')
        year_label.pack()

        self.year_var = StringVar()
        self.year_var.set("2023")

        year_options = [str(year) for year in self.df['startdate'].dt.year.unique()]
        year_menu = OptionMenu(self.root, self.year_var, *year_options)
        year_menu.pack()

        # Attribute selection
        attribute_label = Label(self.root, text='Select Attribute:')
        attribute_label.pack()

        self.attribute_var = StringVar()
        self.attribute_var.set(self.df.columns[2])

        attribute_options = list(self.df.columns[2:])
        attribute_menu = OptionMenu(self.root, self.attribute_var, *attribute_options)
        attribute_menu.pack()

        # Plot type selection
        plot_type_label = Label(self.root, text='Select Plot Type:')
        plot_type_label.pack()

        self.plot_type_var = StringVar()
        self.plot_type_var.set("Time Series")

        plot_type_options = ["Time Series", "Scatter Plot", "Box Plot"]
        plot_type_menu = OptionMenu(self.root, self.plot_type_var, *plot_type_options)
        plot_type_menu.pack()

        button_plot_frame = Frame(self.root)
        button_plot_frame.pack(padx=10, pady=10)

        plot_button = Button(button_plot_frame, text="Plot Data", command=self.update_plot, bg='green', fg='white', font=('Helvetica', 12, 'bold'))
        plot_button.pack(side=LEFT, padx=10)

        # Plot canvas
        self.plot_canvas = FigureCanvasTkAgg(plt.figure(figsize=(8, 6)), master=self.root)
        self.plot_canvas.get_tk_widget().pack(side=LEFT, padx=10, pady=10)

        self.stats_label = Label(self.root, text='', font=('Helvetica', 12, 'bold'))
        self.stats_label.pack(side=LEFT, padx=10, pady=10)

    def update_plot(self):
        selected_year = int(self.year_var.get())
        selected_attribute = self.attribute_var.get()
        selected_plot_type = self.plot_type_var.get()

        df_selected = self.df[self.df['startdate'].dt.year == selected_year]

        plt.clf()

        if selected_plot_type == "Time Series":
            plt.plot(df_selected['startdate'], df_selected[selected_attribute], marker='o')
            plt.xlabel('Date')
            plt.ylabel(selected_attribute)
            plt.title(f'{selected_attribute} in {selected_year}')
            plt.xticks(rotation=45)

        elif selected_plot_type == "Scatter Plot":
            plt.scatter(df_selected[selected_attribute], df_selected['startdate'], marker='o', color='blue')
            plt.ylabel('Date')
            plt.xlabel(selected_attribute)
            plt.title(f'Scatter Plot of {selected_attribute} for {selected_year}')

        elif selected_plot_type == "Box Plot":
            df_selected.boxplot(column=selected_attribute)
            plt.title(f'Box Plot of {selected_attribute} for {selected_year}')

        plt.tight_layout()

        # Update Summary Statistics
        stats_table = df_selected[selected_attribute].describe()
        self.stats_label.config(text=stats_table)

        self.plot_canvas.draw()

# Create the root window and run the GUI application
root = Tk()
app = LagosAirQualityApp(root)
root.mainloop()


FileNotFoundError: [Errno 2] No such file or directory: 'secure-connect-dcbdms-lag.zip'